In [ ]:
# install (quiet to keep logs clean)
!pip -q install ultralytics==8.* roboflow pyspark

# core imports
import os, time, json, glob
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np

import torch
from ultralytics import YOLO

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
# ============================================
# Project Configuration
# ============================================

# ----- experiment configuration -----
RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
PERSON_ONLY = True             # keep only 'person' class (COCO id=0)
IMG_SIZE     = 640
CONF_THRESH  = 0.25
MAX_IMAGES   = None            # set small int while testing (e.g., 100)

# ----- directory structure (based on your project tree) -----
ROOT          = Path.cwd()                           # current project root
DATA_DIR      = ROOT / "data"                        # raw dataset directory
DERIVED_DIR   = ROOT / "derived" / "caltech_yolo"    # derived local data
RUNS_DIR      = ROOT / "runs" / "phase2" / RUN_TAG   # experiment run directory
OUT_IMG_DIR   = RUNS_DIR / "annotated"               # annotated JPGs
OUT_CSV_DIR   = RUNS_DIR / "csv"                     # detection CSVs
FIG_DIR       = ROOT / "reports" / "figures" / RUN_TAG  # plots and visualizations

# make sure all exist
for p in [DATA_DIR, DERIVED_DIR, RUNS_DIR, OUT_IMG_DIR, OUT_CSV_DIR, FIG_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# ----- dataset source -----
# Option A: use existing local dataset (preferred for speed)
# Option B: use Roboflow if you want an on-demand cloud dataset
USE_ROBOFLOW = True

if not USE_ROBOFLOW:
    # Use your derived/caltech_yolo images folder
    DATA_IMG_DIR = DERIVED_DIR / "images" / "test"
else:
    # Roboflow download (expects env var ROBOFLOW_API_KEY)
    from roboflow import Roboflow

    api_key = os.environ.get("ROBOFLOW_API_KEY")
    assert api_key, "Set ROBOFLOW_API_KEY in environment to download dataset securely."

    rf = Roboflow(api_key=api_key)
    project  = rf.workspace("raditya").project("caltech-pedestrian-iitp")
    version  = project.version(1)
    ds       = version.download("yolov11")   # YOLOv11 format images

    # try to find a valid split
    candidates = [
        Path(ds.location) / "valid" / "images",
        Path(ds.location) / "test" / "images",
        Path(ds.location) / "train" / "images",
        Path(ds.location) / "images",   # fallback
    ]
    DATA_IMG_DIR = next((c for c in candidates if c.exists()), None)
    assert DATA_IMG_DIR and any(DATA_IMG_DIR.glob("*.jpg")), f"No images found under {ds.location}"

print(f"\n✅ Directory Structure:")
print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)
print("DERIVED_DIR:", DERIVED_DIR)
print("RUNS_DIR:", RUNS_DIR)
print("FIG_DIR:", FIG_DIR)
print("Images dir:", DATA_IMG_DIR)


In [ ]:
# ============================================
# Load Pretrained Models from Ultralytics Hub
# (no local .pt weights needed)
# ============================================

print("Loading models from Ultralytics Hub...")

# Load directly from Ultralytics registry
yolo_model   = YOLO("yolov8n.pt")    # baseline YOLOv8n
rtdetr_model = YOLO("rtdetr-l.pt")   # RT-DETR Large (Ultralytics implementation)

print("Models loaded successfully.")
print(f"YOLO device: {yolo_model.device}")
print(f"RT-DETR device: {rtdetr_model.device}")

In [ ]:
# ============================================
# Run Inference on Dataset (YOLOv8n & RT-DETR-L)
# ============================================

from tqdm import tqdm

# quick sanity check
assert DATA_IMG_DIR.exists(), f"Dataset path not found: {DATA_IMG_DIR}"
image_files = sorted([p for p in DATA_IMG_DIR.glob("*.jpg")])
if MAX_IMAGES:
    image_files = image_files[:MAX_IMAGES]

print(f"🖼️ Found {len(image_files)} images for inference in {DATA_IMG_DIR}")

# unified inference settings
predict_kwargs = dict(
    imgsz=IMG_SIZE,
    conf=CONF_THRESH,
    save=True,
    save_txt=True,
    save_conf=True,
    project=str(OUT_IMG_DIR),
    name="predictions",
    exist_ok=True,
    verbose=False
)
if PERSON_ONLY:
    predict_kwargs["classes"] = [0]  # COCO 'person'

# helper function for single model inference
def run_inference(model, model_name):
    """Run inference on all images and return a detections DataFrame."""
    rows = []
    print(f"\n🚀 Running inference with {model_name}...")

    for img_path in tqdm(image_files, desc=model_name):
        results = model(img_path, **predict_kwargs, stream=False)
        res = results[0]

        h, w = res.orig_shape
        inf_ms = float(res.speed.get("inference", 0.0)) if hasattr(res, "speed") else 0.0

        for b in res.boxes:
            cls_id = int(b.cls)
            conf = float(b.conf)
            x1, y1, x2, y2 = map(float, b.xyxy[0].tolist())

            rows.append({
                "image_path": str(img_path),
                "model": model_name,
                "class_id": cls_id,
                "class_name": model.names.get(cls_id, str(cls_id)),
                "conf": conf,
                "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                "width": w, "height": h,
                "inf_ms": inf_ms
            })

    df = pd.DataFrame(rows)
    print(f"✅ {model_name}: {len(df)} detections saved.")
    return df


# ---- Run both models ----
df_yolo   = run_inference(yolo_model, "YOLOv8n")
df_rtdetr = run_inference(rtdetr_model, "RT-DETR-L")

# ---- Merge and save ----
df_all = pd.concat([df_yolo, df_rtdetr], ignore_index=True)

out_csv = OUT_CSV_DIR / "phase2_detections.csv"
df_all.to_csv(out_csv, index=False)

print(f"\n📦 All detections saved to: {out_csv.resolve()}")
display(df_all.head())


In [ ]:
def run_model_collect_csv(model, model_name, images, out_img_dir, out_csv_dir,
                          img_size=640, conf=0.25, person_only=True):
    out_img_dir.mkdir(parents=True, exist_ok=True)
    out_csv_dir.mkdir(parents=True, exist_ok=True)

    # where Ultralytics will drop annotated images
    u_project = out_img_dir
    u_name    = f"{model_name}_viz"

    predict_kwargs = dict(
        imgsz=img_size,
        conf=conf,
        save=True,
        save_txt=True,
        save_conf=True,
        project=str(u_project),
        name=u_name,
        exist_ok=True,
        verbose=False,
        stream=True
    )
    if person_only:
        predict_kwargs["classes"] = [0]  # COCO 'person'

    rows = []
    for res in model.predict(source=[str(p) for p in images], **predict_kwargs):
        # per-image timing (ms)
        inf_ms = float(getattr(res, "speed", {}).get("inference", 0.0))
        h, w   = res.orig_shape
        img_p  = res.path
        for b in res.boxes:
            cls_id = int(b.cls)
            conf   = float(b.conf)
            x1, y1, x2, y2 = map(float, b.xyxy[0].tolist())
            rows.append({
                "image_path": img_p,
                "model": model_name,
                "class_id": cls_id,
                "class_name": model.names.get(cls_id, str(cls_id)),
                "conf": conf,
                "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                "width": w, "height": h,
                "inf_ms": inf_ms
            })

    df = pd.DataFrame(rows)
    out_csv = out_csv_dir / f"{model_name}_detections.csv"
    df.to_csv(out_csv, index=False)
    print(f"[{model_name}] boxes: {len(df)} | csv -> {out_csv}")
    return df, out_csv, u_project / u_name

In [ ]:
df_yolo,  csv_yolo,  yolo_run_dir  = run_model_collect_csv(
    yolo_model, "yolov8n", images, OUT_IMG_DIR, OUT_CSV_DIR,
    img_size=IMG_SIZE, conf=CONF_THRESH, person_only=PERSON_ONLY
)

df_rtd,   csv_rtd,   rtd_run_dir   = run_model_collect_csv(
    rtdetr_model, "rtdetr-l", images, OUT_IMG_DIR, OUT_CSV_DIR,
    img_size=IMG_SIZE, conf=CONF_THRESH, person_only=PERSON_ONLY
)

print("YOLO images dir:", yolo_run_dir)
print("RT-DETR images dir:", rtd_run_dir)
display(df_yolo.head(3))
display(df_rtd.head(3))

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, mean, stddev, first

spark = SparkSession.builder.appName("Phase2_Compare").getOrCreate()
print("✅ Spark version:", spark.version)

sdf_yolo = spark.read.option("header", True).csv(str(csv_yolo), inferSchema=True)
sdf_rtd  = spark.read.option("header", True).csv(str(csv_rtd),  inferSchema=True)

# union both with consistent schema (already aligned)
sdf_all = sdf_yolo.unionByName(sdf_rtd)
sdf_all.cache()
print("Rows total:", sdf_all.count())
sdf_all.printSchema()
sdf_all.show(5, truncate=False)

In [ ]:
# overall per-model summary
summary = (sdf_all
           .groupBy("model")
           .agg(
               count("*").alias("num_boxes"),
               mean("conf").alias("avg_conf"),
               stddev("conf").alias("std_conf"),
               mean("inf_ms").alias("avg_inf_ms")
           ))
summary.show()

# detections per image (per model)
per_img = (sdf_all
           .groupBy("model", "image_path")
           .agg(
               count("*").alias("detections"),
               first("inf_ms").alias("inf_ms") # same value per image; take one
           ))
per_img.show(5)

# save spark outputs for reuse
AGG_DIR = RUNS_DIR / "spark_out"
AGG_DIR.mkdir(parents=True, exist_ok=True)
summary.write.mode("overwrite").parquet(str(AGG_DIR / "summary.parquet"))
per_img.write.mode("overwrite").parquet(str(AGG_DIR / "per_image.parquet"))
print("Saved parquet to:", AGG_DIR)

In [ ]:
pdf_summary = summary.toPandas().sort_values("model")
pdf_perimg  = per_img.toPandas()

# simple sanity prints
display(pdf_summary)
display(pdf_perimg.head())

# 1) avg confidence by model
plt.figure(figsize=(6,4))
sns.barplot(x="model", y="avg_conf", data=pdf_summary)
plt.title("Average detection confidence")
plt.ylabel("Avg confidence")
plt.xlabel("")
plt.tight_layout()
plt.savefig(FIG_DIR / "avg_conf_by_model.png", dpi=200)
plt.show()

# 2) avg per-image latency by model
plt.figure(figsize=(6,4))
sns.barplot(x="model", y="avg_inf_ms", data=pdf_summary)
plt.title("Average per-image inference time (ms)")
plt.ylabel("Avg inference (ms)")
plt.xlabel("")
plt.tight_layout()
plt.savefig(FIG_DIR / "avg_inf_ms_by_model.png", dpi=200)
plt.show()

# 3) detections per image distribution (overlayed)
plt.figure(figsize=(7,4))
for m, g in pdf_perimg.groupby("model"):
    sns.kdeplot(g["detections"], label=m)
plt.title("Detections per image (KDE)")
plt.xlabel("# detections")
plt.legend()
plt.tight_layout()
plt.savefig(FIG_DIR / "detections_per_image_kde.png", dpi=200)
plt.show()

# 4) confidence histogram (sample for scale)
pdf_all = sdf_all.sample(fraction=0.25, seed=7).toPandas() if sdf_all.count() > 10000 else sdf_all.toPandas()
plt.figure(figsize=(7,4))
sns.histplot(data=pdf_all, x="conf", hue="model", bins=30, stat="density", element="step", common_norm=False)
plt.title("Confidence distribution")
plt.tight_layout()
plt.savefig(FIG_DIR / "confidence_hist.png", dpi=200)
plt.show()

print("Figures ->", FIG_DIR)


In [ ]:
import shutil

picked = []
for model_name, run_dir in [("yolov8n", yolo_run_dir), ("rtdetr-l", rtd_run_dir)]:
    samples = sorted(run_dir.glob("*.jpg"))[:3]
    for s in samples:
        dst = FIG_DIR / f"{model_name}_{s.name}"
        shutil.copy2(s, dst)
        picked.append(dst)

print("Sample annotated images copied:")
for p in picked:
    print(" -", p)
